# Import

In [ ]:
%run init.ipynb

# Bitcoin

Idea: use the new price data to extend the current data

## Suply

In [ ]:
with open("../../temp/total-bitcoins.json") as f:
    total_btc = json.load(f)["total-bitcoins"]
total_btc[:3]

In [ ]:
total_btc = pd.DataFrame(total_btc)
total_btc.head(3)

In [ ]:
total_btc = pd.DataFrame({
    "ts":(total_btc["x"]/10**3).map(datetime.fromtimestamp),
    "n_coins":total_btc["y"]
}).set_index("ts").squeeze()
total_btc.head(3)

In [ ]:
total_btc = total_btc.resample("ME").last()
total_btc

In [ ]:
dump_pickle(total_btc, "total_btc")

## Volume

### The Block

In [ ]:
block_data = pd.read_csv(
    "../../temp/Volume_of_Bitcoin_Futures_(BTC_not_USD)_2025_12_18.csv",
    index_col="dt",
    parse_dates=True,
    date_format="%d/%m/%y %H:%M"
)
block_data.head(3)

In [ ]:
block_data = block_data.resample("ME")["volume"].sum()
block_data.head(3)

In [ ]:
dump_pickle(block_data, "btc_volume_block")

### CMEG

* Downloaded data manually
* Starts on 2013-01

In [ ]:
file_exists = pd.Series(
    False, 
    index=pd.date_range(start="2013-01-01", end="2025-08-01", freq="ME").strftime("%Y%m") + ".zip"
)
file_exists

In [ ]:
for i in file_exists.index.copy():
    file_exists.loc[i] = Path(f"../../temp/{i}").exists()
file_exists

In [ ]:
file_exists[file_exists == False]
# all files are present

In [ ]:
expected_files = list(
    "CMEG_" + pd.date_range(start="2013-01-01", end="2025-08-01", freq="ME").strftime("%Y%m") + ".pdf"
)
expected_files[:3]

In [ ]:
# ignore dot files
got_files = list(i.name for i in Path("../../temp/").iterdir() if not i.name.startswith("."))
got_files[:3]

In [ ]:
assert len(expected_files) == len(got_files), f"The number of actual files does not equal what is expected. Expected: {len(expected_files)}, got {len(got_files)}"

assert len(got_files) == len(set(got_files)), "There are duplicated in fetched files"

for exp, got in zip(sorted(expected_files), sorted(got_files)):
    assert exp == got, f"File names don't match: {exp} vs. {got}"
else:
    print(f"All expected files have been fetched and processed!")

In [ ]:
srs = pd.Series()

for file in Path("../../temp/").iterdir():
    if not file.name.startswith("CMEG"):
        continue

    for name in ["BITCOIN FUTURES", "BITCOIN"]:            
        try:
            volume = extract_asset_volume(
                path=file,
                asset=name
            )
            break
        except ValueError:
            volume = -1
    srs.loc[datetime(int(file.name[5:9]), int(file.name[9:11]), 1)] = volume
    print(f"{datetime(int(file.name[5:9]), int(file.name[9:11]), 1)}: {volume}")

srs = srs.sort_index()
srs[srs == -1] = np.nan
srs = srs * 5
srs

In [ ]:
dump_pickle(srs, "CMEG_volume")

## Concat

In [ ]:
block_volume = load_pickle("btc_volume_block")
block_volume.index.name = ""
block_volume

In [ ]:
cmeg_volume = load_pickle("CMEG_volume")
cmeg_volume = cmeg_volume.resample("ME").last()
cmeg_volume

In [ ]:
total_volume = (block_volume + cmeg_volume).dropna()
total_volume

In [ ]:
total_btc = load_pickle("total_btc")
total_btc

In [ ]:
df = pd.concat(
    [total_btc, total_volume],
    axis="columns",
    join="outer"
)
df = df.rename(columns={"n_coins":"supply", 0:"trading_volume"})
df = df.reindex(
    pd.date_range("2010-01-01", "2025-08-01", freq="ME")
)
df = df.dropna(axis="rows", how="any")
df

In [ ]:
df = df.reset_index()
df["index"] = "'" + df["index"].dt.strftime("%F") + "'"
df

In [ ]:
dry_insert_into_db(
    conn_str=CONNECTION_STRING, 
    table="bitcoin_trading_metadata",
    data=df.to_numpy()
)

In [ ]:
insert_into_db(
    conn_str=CONNECTION_STRING, 
    table="bitcoin_trading_metadata",
    data=df.to_numpy()
)

In [ ]:
logger.info("Bitcoin supply and demand have been acquired and inserted")

# S&P 500

## Supply

* downloaded data manually from the source

In [ ]:
df = pd.read_excel(
    io = "../../temp/snp_500_data.xlsx",
    sheet_name = "Historisch",
    header = 0,
    usecols = ["per", "Umlaufende Anteile"],
    na_values = "--",
    thousands = ",",
    decimal = "."
)
df.head(3)

In [ ]:
# split string dates into day, month, year
# some have form "08.Okt.2025", some are missing a full stop after the month "14.März2025"
# So we:
#     match the day into the first group, 
#     then skip the point, 
#     match month, 
#     optional point, 
#     match the year
pattern = re.compile(r"(\d{1,2})\.(Jan|Feb|März|Apr|Mai|Juni|Juli|Aug|Sept|Okt|Nov|Dez)\.?(\d{4})")
month_mapping = {k:v for k, v in zip("Jan|Feb|März|Apr|Mai|Juni|Juli|Aug|Sept|Okt|Nov|Dez".split("|"),range(1,13))}
dates = []

for i in df["per"]:
    match_object = re.match(pattern, i)
    group = list(match_object.groups())
    group[1] = month_mapping[group[1]]
    group = list(map(int, group))
    group = group[::-1]

    dates.append(
        datetime(*group)
    )
dates[:3]

In [ ]:
df["per"] = dates
df = df.rename(columns={"per":"dt", "Umlaufende Anteile": "outstanding_supply"})
df = df.set_index("dt")
df = df.squeeze().sort_index()
df

In [ ]:
# the supply does show significant downward fluctuations.
# Thus, monthly average will be taken
df.plot(kind="line")

In [ ]:
df = (df
    .resample("ME")
    .mean()
    .loc["2010-01-01":"2025-08-01"]
    .copy()
)
df

In [ ]:
df.plot(kind="line")

In [ ]:
dump_pickle(
    df,
    "snp_aggregated_supply_data"
)

## Volume

* Manually save the [source page](https://www.ishares.com/uk/individual/en/products/253743/ishares-sp-500-b-ucits-etf-acc-fund) into temp (HTML only)

In [ ]:
TICKER_TYPE = "Bloomberg Ticker"

with open("../../temp/iShares Core S&P 500 UCITS ETF _ CSPX.html") as f:
    html_dump = f.read()

soup = BeautifulSoup(html_dump, "html.parser")
table = soup.find("table",attrs={"id":"listingsTable"})
df = pd.read_html(
    StringIO(str(table)),
    na_values="-"
)[0]

del html_dump, soup, table

na_exchanges = list(df.loc[df[TICKER_TYPE].isna(), "Exchange"])
yf_tickers = list(df[TICKER_TYPE].dropna())

df

In [ ]:
yf_data = yf.download(
    tickers=list(df["RIC"].dropna()) + ["CSPX.AS", "CSSPX.SW"],
    interval="1mo",
    start="2010-01-01",
    end="2025-08-01",
    group_by="Ticker",
    keepna=True
)

yf_data = yf_data.xs(
    "Volume", 
    axis="columns", 
    level=1
)

trading_volume = yf_data.dropna(how="all").sum(axis="columns") / 0.93
trading_volume = trading_volume.resample("ME").last()
trading_volume

Missing exchanges:
* Bolsa De Valores De Colombia
* Santiago Stock Exchange
* Tel Aviv Stock Exchange

Together ~7% of the listed exchanges. Source: ChatGPT

In [ ]:
dump_pickle(trading_volume, "snp_trading_volume")

## Concat

In [ ]:
supply = load_pickle("snp_aggregated_supply_data")
volume = load_pickle("snp_trading_volume")

In [ ]:
df = pd.concat(
    [supply, volume],
    axis="columns",
    join="outer"
)
df = df.rename(columns={0:"volume"})

dates = pd.date_range(start="2010-01-01", end="2025-08-01", freq="ME")
df.reindex(dates).fillna(0)

In [ ]:
# validate index
pd.Series(df.index).is_monotonic_increasing

In [ ]:
df = df.reset_index()
df["index"] = "'" + df["index"].astype(str) + "'"
df

In [ ]:
dry_insert_into_db(
    CONNECTION_STRING,
    "snp_trading_metadata",
    df.to_numpy()
)

In [ ]:
insert_into_db(
    CONNECTION_STRING,
    "snp_trading_metadata",
    df.to_numpy()
)

In [ ]:
logger.info("S&P 500 supply and demand have been acquired and inserted")

# NASDAQ

## Supply

* downloaded data manually from the source

In [ ]:
df = pd.read_excel(
    io = "../../temp/nasdaq_data.xlsx",
    sheet_name = "Historical",
    header = 0,
    usecols = ["As Of", "Securities In Issue"],
    na_values = "--",
    thousands = ",",
    decimal = "."
)
df.head(3)

In [ ]:
# split string dates into day, month, year
# some have form "08.Okt.2025", some are missing a full stop after the month "14.März2025"
# So we:
#     match the day into the first group, 
#     then skip the point, 
#     match month, 
#     optional point, 
#     match the year
pattern = re.compile(r"(\d{1,2})\/(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec)\/?(\d{4})")
month_mapping = {k:v for k, v in zip("Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec".split("|"),range(1,13))}
dates = []

for i in df["As Of"]:
    match_object = re.match(pattern, i)
    group = list(match_object.groups())
    group[1] = month_mapping[group[1]]
    group = list(map(int, group))
    group = group[::-1]

    dates.append(
        datetime(*group)
    )
dates[:3]

In [ ]:
df["As Of"] = dates
df = df.rename(columns={"As Of":"dt", "Securities In Issue": "outstanding_supply"})
df = df.set_index("dt")
df = df.squeeze().sort_index()
df

In [ ]:
# the supply does show significant downward fluctuations.
# Thus, monthly average will be taken
df.plot(kind="line")

In [ ]:
df = (df
    .resample("ME")
    .mean()
    .loc["2010-01-01":"2025-08-01"]
    .copy()
)
df

In [ ]:
df.plot(kind="line")

In [ ]:
dump_pickle(
    df,
    "nasdaq_aggregated_supply_data"
)

## Volume

In [ ]:
TICKER_TYPE = "Bloomberg Ticker"

with open("../../temp/iShares NASDAQ 100 UCITS ETF _ CNDX.html") as f:
    html_dump = f.read()

soup = BeautifulSoup(html_dump, "html.parser")
table = soup.find("table",attrs={"id":"listingsTable"})
df = pd.read_html(
    StringIO(str(table)),
    na_values="-"
)[0]

del html_dump, soup, table

na_exchanges = list(df.loc[df[TICKER_TYPE].isna(), "Exchange"])
yf_tickers = list(df[TICKER_TYPE].dropna())

df

* Tel Aviv is missing

In [ ]:
yf_data = yf.download(
    tickers=list(df["RIC"].dropna()) + ["CNDX.AS", "CSNDX.SW"],
    interval="1mo",
    start="2010-01-01",
    end="2025-08-01",
    group_by="Ticker",
    keepna=True
)

yf_data = yf_data.xs(
    "Volume", 
    axis="columns", 
    level=1
)

trading_volume = yf_data.dropna(how="all").sum(axis="columns") / 0.93
trading_volume = trading_volume.resample("ME").last()
trading_volume

In [ ]:
dump_pickle(
    trading_volume,
    "nasdaq_trading_volume"
)

## Concat

In [ ]:
supply = load_pickle("nasdaq_aggregated_supply_data")
trading_volume = load_pickle("nasdaq_trading_volume")

In [ ]:
supply

In [ ]:
trading_volume

In [ ]:
df = pd.concat(
    [supply, trading_volume],
    axis="columns",
    join="outer"
)
df = df.rename(columns={0:"trading_volume"})
df.head(3)

In [ ]:
df = df.reset_index()
df["index"] = "'" + df["index"].astype("str") + "'"
df

In [ ]:
dry_insert_into_db(
    CONNECTION_STRING,
    "nasdaq_trading_metadata",
    df.to_numpy()
)

In [ ]:
insert_into_db(
    CONNECTION_STRING,
    "nasdaq_trading_metadata",
    df.to_numpy()
)

In [ ]:
logger.info("Successfully inserted NASDAQ trading metadata")

# Dow Jones

## Supply

* downloaded data manually from the source

In [ ]:
df = pd.read_excel(
    io = "../../temp/dow_jones_data.xlsx",
    sheet_name = "Historical",
    header = 0,
    usecols = ["As Of", "Securities In Issue"],
    na_values = "--",
    thousands = ",",
    decimal = "."
)
df.head(3)

In [ ]:
# split string dates into day, month, year
# some have form "08.Okt.2025", some are missing a full stop after the month "14.März2025"
# So we:
#     match the day into the first group, 
#     then skip the point, 
#     match month, 
#     optional point, 
#     match the year
pattern = re.compile(r"(\d{1,2})\/(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec)\/?(\d{4})")
month_mapping = {k:v for k, v in zip("Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec".split("|"),range(1,13))}
dates = []

for i in df["As Of"]:
    match_object = re.match(pattern, i)
    group = list(match_object.groups())
    group[1] = month_mapping[group[1]]
    group = list(map(int, group))
    group = group[::-1]

    dates.append(
        datetime(*group)
    )
dates[:3]

In [ ]:
df["As Of"] = dates
df = df.rename(columns={"As Of":"dt", "Securities In Issue": "outstanding_supply"})
df = df.set_index("dt")
df = df.squeeze().sort_index()
df

In [ ]:
df = (df
    .resample("ME")
    .mean()
    .loc["2010-01-01":"2025-08-01"]
    .copy()
)
df

In [ ]:
dump_pickle(
    df,
    "dow_jones_aggregated_supply_data"
)

## Volume

In [ ]:
TICKER_TYPE = "Bloomberg Ticker"

with open("../../temp/iShares Dow Jones Industrial Average UCITS ETF _ CIND.html") as f:
    html_dump = f.read()

soup = BeautifulSoup(html_dump, "html.parser")
table = soup.find("table",attrs={"id":"listingsTable"})
df = pd.read_html(
    StringIO(str(table)),
    na_values="-"
)[0]

del html_dump, soup, table

na_exchanges = list(df.loc[df[TICKER_TYPE].isna(), "Exchange"])
yf_tickers = list(df[TICKER_TYPE].dropna())

df

In [ ]:
yf_data = yf.download(
    tickers=list(df["RIC"].dropna()) + ["CSINDU.SW"],
    interval="1mo",
    start="2010-01-01",
    end="2025-08-01",
    group_by="Ticker",
    keepna=True
)

yf_data = yf_data.xs(
    "Volume", 
    axis="columns", 
    level=1
)

trading_volume = yf_data.dropna(how="all").sum(axis="columns") / 0.93
trading_volume = trading_volume.resample("ME").last()
trading_volume

In [ ]:
dump_pickle(
    trading_volume,
    "dow_jones_trading_volume"
)

## Concat

In [ ]:
supply = load_pickle("dow_jones_aggregated_supply_data")
trading_volume = load_pickle("dow_jones_trading_volume")

In [ ]:
supply

In [ ]:
trading_volume

In [ ]:
df = pd.concat(
    [supply, trading_volume],
    axis="columns",
    join="outer"
)
df = df.rename(columns={0:"trading_volume"})
df.head(3)

In [ ]:
df = df.reset_index()
df["index"] = "'" + df["index"].astype("str") + "'"
df

In [ ]:
dry_insert_into_db(
    CONNECTION_STRING,
    "dow_jones_trading_metadata",
    df.to_numpy()
)

In [ ]:
insert_into_db(
    CONNECTION_STRING,
    "dow_jones_trading_metadata",
    df.to_numpy()
)

# Gold

## Supply

In [ ]:
total_supply = pd.read_excel(
    "../../temp/GDT_Tables_Q425_EN.xlsx",
    sheet_name="Gold Balance"
).iloc[4,22:-3]
total_supply = total_supply.astype(float)
total_supply.index = pd.date_range(start="2010-01-01", freq="QE", periods=63)
total_supply = total_supply * 1000
total_supply

In [ ]:
dump_pickle(
    total_supply,
    "gold_aggregated_supply"
)

## Volume

### CMEG

* Downloaded data manually
* Starts on 2013-01

In [ ]:
file_exists = pd.Series(
    False, 
    index=pd.date_range(start="2013-01-01", end="2025-08-01", freq="ME").strftime("%Y%m") + ".zip"
)
file_exists

In [ ]:
for i in file_exists.index.copy():
    file_exists.loc[i] = Path(f"../../temp/{i}").exists()
file_exists

In [ ]:
file_exists[file_exists == False]
# all files are present

In [ ]:
expected_files = list(
    "CMEG_" + pd.date_range(start="2013-01-01", end="2025-08-01", freq="ME").strftime("%Y%m") + ".pdf"
)
expected_files[:3]

In [ ]:
# ignore dot files
got_files = list(i.name for i in Path("../../temp/").iterdir() if not i.name.startswith("."))
got_files[:3]

In [ ]:
assert len(expected_files) == len(got_files), f"The number of actual files does not equal what is expected. Expected: {len(expected_files)}, got {len(got_files)}"

assert len(got_files) == len(set(got_files)), "There are duplicated in fetched files"

for exp, got in zip(sorted(expected_files), sorted(got_files)):
    assert exp == got, f"File names don't match: {exp} vs. {got}"
else:
    print(f"All expected files have been fetched and processed!")

In [ ]:
srs = pd.Series()

for file in Path("../../temp/").iterdir():
    if not file.name.startswith("CMEG"):
        continue

    for name in ["COMEX GOLD"]:            
        try:
            volume = extract_asset_volume(
                path=file,
                asset=name
            )
            break
        except ValueError:
            volume = -1
    srs.loc[datetime(int(file.name[5:9]), int(file.name[9:11]), 1)] = volume
    print(f"{datetime(int(file.name[5:9]), int(file.name[9:11]), 1)}: {volume}")

srs = srs.sort_index()
srs

In [ ]:
# manual sanity check for year 2022
# since there are multiple COMEX GOLD entries. Luckily we only need the first one
srs[srs.index.year==2022]

In [ ]:
# Convert units
srs = (1 / 0.311034768) * srs
srs

In [ ]:
dump_pickle(srs, "CMEG_gold_volume")

### Shanghai exchange

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.shfe.com.cn/eng/reports/StatisticalData/MonthlyData/?query_params=month")

In [ ]:
data = []

In [ ]:
def wait_dom_stable(driver, *, min_wait=1, stable_for=1, timeout=15.0, poll=0.1):
    """
    Wait until driver.page_source stops changing for `stable_for` seconds,
    but never return before `min_wait` seconds have elapsed.
    """
    start = time.time()
    end = start + timeout

    last = driver.page_source
    last_change = start

    while True:
        now = time.time()
        if now >= end:
            raise TimeoutError("DOM did not stabilize within timeout")

        time.sleep(poll)
        cur = driver.page_source

        if cur != last:
            last = cur
            last_change = time.time()

        now = time.time()
        # require both: (a) we've waited at least min_wait and (b) stable window reached
        if (now - start) >= min_wait and (now - last_change) >= stable_for:
            return

xpath = (
        "//button"
        "[contains(@class,'el-button') "
        "and contains(@class,'el-button--plain') "
        "and contains(@class,'el-button--mini') "
        "and not(@disabled) "
        "and not(contains(@class,'is-disabled')) "
        "and (.//*[normalize-space()='‹' or contains(normalize-space(),'‹')] "
        "     or contains(normalize-space(),'‹'))]"
    )
wait = WebDriverWait(driver, 15)

In [ ]:
for _ in range(187):
    wait_dom_stable(driver)
    sleep(random.uniform(1,2))
    
    try:
        tables = pd.read_html(StringIO(driver.page_source))
    except Exception:
        tables = "Error"
    
    try:
        month = driver.find_element(By.CLASS_NAME, "el-calendar__title").get_attribute("textContent").strip()
    except Exception:
        month = "Error"

    data.append(
        (month, tables)
    )

    # Skip to the next month
    
    # wait for at least one matching button
    wait.until(lambda d: len(d.find_elements(By.XPATH, xpath)) > 0)
    
    # choose the first one that is actually displayed + enabled
    btn = next(
        (b for b in driver.find_elements(By.XPATH, xpath)
         if b.is_displayed() and b.is_enabled()),
        None
    )
    
    # bring into view (centered tends to avoid sticky overlays)
    driver.execute_script(
        "arguments[0].scrollIntoView({block:'center', inline:'center'});", btn
    )
    
    try:
        btn.click()
    except ElementClickInterceptedException:
        # common when an overlay intercepts the click; try moving to it then clicking
        ActionChains(driver).move_to_element(btn).pause(0.1).click().perform()
    except AttributeError:
        print("Cannot continue iteration. Button was not found")
        input("Enter to continue")

In [ ]:
dump_pickle(
    data,
    "raw_scraped_schanghai_data"
)

In [ ]:
driver.close()

---

In [ ]:
data = load_pickle("raw_scraped_schanghai_data")

In [ ]:
len(data)

In [ ]:
srs = pd.Series(
    index=pd.date_range(start="2010-01-01", end="2025-07-31", freq="MS")
)

problematic_entries = []

for (_, i) in data:
    date = datetime.fromisoformat(i[3].columns[0].split()[1] + "-01")
    try:
        volume = i[24].iloc[-1,9]
    except Exception as err:
        if "index 9 is out of bounds for axis 0 with size 6" == str(err):
            #print(str(err))
            volume = i[14].iloc[-1,9]
        elif "list index out of range" == str(err):
            volume = i[12].iloc[-1, 9]
        else:
            print(f"{type(err)} - {err}")
            problematic_entries.append((date, i))
            volume = np.nan
    
    srs.loc[date] = volume    

srs

In [ ]:
srs[srs.isna()].sort_index(ascending=False)

In [ ]:
dump_pickle(srs, "shanghai_gold_volume")

### TOCOM

In [ ]:
pd.read_excel("../../temp/CDF_M_202007.xlsx").iloc[5:11,-8].sum()